# PASO 1 - Obtención de datos de Google Trends (Automatizado con Pytrends)

Este cuaderno tiene como objetivo recolectar datos de tendencias de búsqueda a nivel global o regional de forma **automatizada**.  
Utilizaremos la librería [Pytrends](https://github.com/GeneralMills/pytrends) para interactuar con la API no oficial de Google Trends y descargar los resultados en formato CSV, dejando todo **listo** para el análisis posterior.

---

## 1. Instalación de librerías necesarias

Primero, nos aseguramos de instalar/actualizar Pytrends en caso de no tenerlo.  
Si estás en Google Colab, Jupyter local u otro entorno, ejecuta la siguiente celda.

In [3]:
%%capture
%pip install --upgrade pytrends

## 2. Importación de librerías y configuración inicial

- **Pandas**: Para manipular y guardar los datos en CSV.  
- **Pytrends**: Para conectarnos a Google Trends.  
- Configuramos la zona horaria y el idioma deseado.  

In [4]:
import os
import time
import pandas as pd
from pytrends.request import TrendReq

# =====================================
# Configuración inicial de Pytrends
# =====================================
# hl: lenguaje (por ejemplo, 'en-US', 'es-ES')
# tz: zona horaria (offset en minutos del UTC)
pytrends = TrendReq(hl='en-US', tz=360)

## 3. Definición de parámetros de búsqueda

En esta sección establecemos:
- Lista de **palabras clave** (o tópicos) que queremos investigar.
- Rango de tiempo de interés, por ejemplo 'today 12-m' (últimos 12 meses).
- Ubicación geográfica (geo), que puede ser '' para "Todo el mundo" o un código de país, p.ej. 'US', 'ES', etc.

**Nota:** Para este "PASO 1", haremos una búsqueda inicial y amplia con un conjunto de keywords genéricos para explorar tendencias globales.

In [5]:
# Lista de palabras clave para la exploración
keywords = [
    'shopping',
    'fitness',
    'technology',
    'beauty',
    'health'
]

# Opciones de timeframe y ubicación
timeframe = 'today 12-m'   # Últimos 12 meses
geo = ''                   # '' = Worldwide, también puedes usar 'ES' (España), 'US' (Estados Unidos), etc.

# Carpetas para guardar datos
DATA_FOLDER = '../data'
if not os.path.exists(DATA_FOLDER):
    os.makedirs(DATA_FOLDER)

## 4. Construcción de la "payload" y obtención de datos de "Interest Over Time"

Con Pytrends, cada vez que construimos una *payload*, especificamos las keywords, el rango de tiempo y la región.
Luego, podemos obtener:
- **Interest over time**: los niveles de interés a lo largo del tiempo.
- **Related topics**: temas relacionados que la gente busca junto a nuestras keywords.
- **Related queries**: consultas relacionadas (búsquedas específicas).
- **Interest by region**: para ver la distribución geográfica del interés.

In [6]:
# Paso 4.1: Construir la payload de Pytrends
pytrends.build_payload(
    kw_list=keywords,
    timeframe=timeframe,
    geo=geo
)

# Paso 4.2: Obtener el "Interest Over Time"
interest_over_time_df = pytrends.interest_over_time()

# Revisamos un pequeño vistazo al dataframe
print("Interest over time (primeras 5 filas):")
display(interest_over_time_df.head())

# Guardamos en CSV
iot_csv_path = os.path.join(DATA_FOLDER, 'interest_over_time.csv')
interest_over_time_df.to_csv(iot_csv_path, index=True)
print(f"Archivo guardado: {iot_csv_path}")

Interest over time (primeras 5 filas):


/Users/josephcode/.pyenv/versions/3.10.13/lib/python3.10/site-packages/pytrends/request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)


,shopping,fitness,technology,beauty,health,isPartial
date,,,,,,
2023-12-24,43,31,15,41,64,False
2023-12-31,39,38,17,39,83,False
2024-01-07,34,33,20,36,92,False
2024-01-14,33,32,20,36,89,False
2024-01-21,33,31,22,36,90,False


Archivo guardado: ../data/interest_over_time.csv


## 5. Related Topics y Related Queries

Estas funciones proveen información de los temas y búsquedas que Google asocia con nuestras keywords.  
Podremos descubrir tendencias o conceptos emergentes que no habíamos considerado.

In [10]:
# Paso 5.1: Related Topics
# Esto devuelve un diccionario cuyas keys son cada keyword,
# y el contenido son dataframes de "top" y "rising" topics.

# Configurar Pytrends con reintentos y backoff
pytrends = TrendReq(
    hl='en-US',
    tz=360,
    retries=5,
    backoff_factor=0.1
)

DATA_FOLDER = './data'
os.makedirs(DATA_FOLDER, exist_ok=True)

keywords = ['shopping', 'fitness', 'technology']

for kw in keywords:
    # 1. Construir payload
    pytrends.build_payload(
        kw_list=[kw],
        timeframe='today 5-y',
        geo='US'
    )

    # Manejar el caso en que Google no devuelva 'rankedList'
    try:
        related_topics_dict = pytrends.related_topics()
    except IndexError:
        print(f"[WARNING] Google no devolvió datos de 'rankedList' para '{kw}'. Skipping...")
        continue

    # 2. Procesar 'top' y 'rising' solo si existen
    if kw in related_topics_dict:
        # 'top'
        if 'top' in related_topics_dict[kw] and not related_topics_dict[kw]['top'].empty:
            df_top_topics = related_topics_dict[kw]['top']
            df_top_path = os.path.join(DATA_FOLDER, f'related_topics_top_{kw}.csv')
            df_top_topics.to_csv(df_top_path, index=False)
            print(f"Related TOP topics para '{kw}' guardados en: {df_top_path}")
        else:
            print(f"Sin datos de 'top' para '{kw}'")

        # 'rising'
        if 'rising' in related_topics_dict[kw] and not related_topics_dict[kw]['rising'].empty:
            df_rising_topics = related_topics_dict[kw]['rising']
            df_rising_path = os.path.join(DATA_FOLDER, f'related_topics_rising_{kw}.csv')
            df_rising_topics.to_csv(df_rising_path, index=False)
            print(f"Related RISING topics para '{kw}' guardados en: {df_rising_path}")
        else:
            print(f"Sin datos de 'rising' para '{kw}'")
    else:
        print(f"[WARNING] '{kw}' no está en 'related_topics_dict'. No hay datos.")

    # 3. Related Queries
    # (mismo tipo de verificación)
    try:
        related_queries_dict = pytrends.related_queries()
    except IndexError:
        print(f"[WARNING] Google no devolvió queries para '{kw}'. Skipping...")
        continue

    if kw in related_queries_dict:
        if 'top' in related_queries_dict[kw] and not related_queries_dict[kw]['top'].empty:
            df_top_queries = related_queries_dict[kw]['top']
            df_topq_path = os.path.join(DATA_FOLDER, f'related_queries_top_{kw}.csv')
            df_top_queries.to_csv(df_topq_path, index=False)
            print(f"Related TOP queries para '{kw}' guardados en: {df_topq_path}")
        else:
            print(f"Sin datos de 'top queries' para '{kw}'")

        if 'rising' in related_queries_dict[kw] and not related_queries_dict[kw]['rising'].empty:
            df_rising_queries = related_queries_dict[kw]['rising']
            df_risingq_path = os.path.join(DATA_FOLDER, f'related_queries_rising_{kw}.csv')
            df_rising_queries.to_csv(df_risingq_path, index=False)
            print(f"Related RISING queries para '{kw}' guardados en: {df_risingq_path}")
        else:
            print(f"Sin datos de 'rising queries' para '{kw}'")
    else:
        print(f"[WARNING] '{kw}' no está en 'related_queries_dict'. No hay datos.")

    time.sleep(5)  # Pausa para evitar bloqueos

[WARNING] Google no devolvió datos de 'rankedList' para 'shopping'. Skipping...
[WARNING] Google no devolvió datos de 'rankedList' para 'fitness'. Skipping...
[WARNING] Google no devolvió datos de 'rankedList' para 'technology'. Skipping...


## **Ampliación: Identificar países con mayor interés y profundizar en Related Topics**

In [16]:
import os
import time
import pandas as pd
from pytrends.request import TrendReq

# ======================================================
# 1. CONFIGURACIÓN BÁSICA DE PYTRENDS
# ======================================================
pytrends = TrendReq(
    hl='en-US',         # Idioma: en-US, es-ES, etc.
    tz=360,             # Zona horaria (ej. UTC-6 = 360)
    retries=5,          # Reintentos en caso de error
    backoff_factor=0.1  # Pausa incremental entre reintentos
)

# Carpeta para guardar CSV
DATA_FOLDER = 'data'
os.makedirs(DATA_FOLDER, exist_ok=True)

# Lista de keywords a analizar
keywords = ['shopping', 'fitness', 'technology']

# Rango de tiempo a nivel mundial
# Si deseas más datos, usa 'today 5-y'
TIMEFRAME_GLOBAL = 'today 12-m'

# ======================================================
# 2. DICCIONARIO DE MAPEO: NOMBRE -> CÓDIGO ISO
# ======================================================
# Incluye todos los países listados
country_code_map = {
    'Afghanistan': 'AF',
    'Albania': 'AL',
    'Algeria': 'DZ',
    'Andorra': 'AD',
    'Angola': 'AO',
    'Antigua & Barbuda': 'AG',
    'Argentina': 'AR',
    'Armenia': 'AM',
    'Australia': 'AU',
    'Austria': 'AT',
    'Azerbaijan': 'AZ',
    'Bahamas': 'BS',
    'Bahrain': 'BH',
    'Bangladesh': 'BD',
    'Barbados': 'BB',
    'Belarus': 'BY',
    'Belgium': 'BE',
    'Belize': 'BZ',
    'Benin': 'BJ',
    'Bhutan': 'BT',
    'Bolivia': 'BO',
    'Bosnia & Herzegovina': 'BA',
    'Botswana': 'BW',
    'Brazil': 'BR',
    'Brunei': 'BN',
    'Bulgaria': 'BG',
    'Burkina Faso': 'BF',
    'Burundi': 'BI',
    'Cabo Verde': 'CV',
    'Cambodia': 'KH',
    'Cameroon': 'CM',
    'Canada': 'CA',
    'Central African Republic': 'CF',
    'Chad': 'TD',
    'Chile': 'CL',
    'China': 'CN',
    'Colombia': 'CO',
    'Comoros': 'KM',
    'Congo (Congo-Brazzaville)': 'CG',
    'Congo (DRC)': 'CD',
    'Costa Rica': 'CR',
    'Cote d’Ivoire': 'CI',
    'Croatia': 'HR',
    'Cuba': 'CU',
    'Cyprus': 'CY',
    'Czechia': 'CZ',
    'Denmark': 'DK',
    'Djibouti': 'DJ',
    'Dominica': 'DM',
    'Dominican Republic': 'DO',
    'Ecuador': 'EC',
    'Egypt': 'EG',
    'El Salvador': 'SV',
    'Equatorial Guinea': 'GQ',
    'Eritrea': 'ER',
    'Estonia': 'EE',
    'Eswatini': 'SZ',
    'Ethiopia': 'ET',
    'Fiji': 'FJ',
    'Finland': 'FI',
    'France': 'FR',
    'Gabon': 'GA',
    'Gambia': 'GM',
    'Georgia': 'GE',
    'Germany': 'DE',
    'Ghana': 'GH',
    'Greece': 'GR',
    'Grenada': 'GD',
    'Guatemala': 'GT',
    'Guinea': 'GN',
    'Guinea-Bissau': 'GW',
    'Guyana': 'GY',
    'Haiti': 'HT',
    'Honduras': 'HN',
    'Hungary': 'HU',
    'Iceland': 'IS',
    'India': 'IN',
    'Indonesia': 'ID',
    'Iran': 'IR',
    'Iraq': 'IQ',
    'Ireland': 'IE',
    'Israel': 'IL',
    'Italy': 'IT',
    'Jamaica': 'JM',
    'Japan': 'JP',
    'Jordan': 'JO',
    'Kazakhstan': 'KZ',
    'Kenya': 'KE',
    'Kiribati': 'KI',
    'Kuwait': 'KW',
    'Kyrgyzstan': 'KG',
    'Laos': 'LA',
    'Latvia': 'LV',
    'Lebanon': 'LB',
    'Lesotho': 'LS',
    'Liberia': 'LR',
    'Libya': 'LY',
    'Liechtenstein': 'LI',
    'Lithuania': 'LT',
    'Luxembourg': 'LU',
    'Madagascar': 'MG',
    'Malawi': 'MW',
    'Malaysia': 'MY',
    'Maldives': 'MV',
    'Mali': 'ML',
    'Malta': 'MT',
    'Marshall Islands': 'MH',
    'Mauritania': 'MR',
    'Mauritius': 'MU',
    'Mexico': 'MX',
    'Micronesia': 'FM',
    'Moldova': 'MD',
    'Monaco': 'MC',
    'Mongolia': 'MN',
    'Montenegro': 'ME',
    'Morocco': 'MA',
    'Mozambique': 'MZ',
    'Myanmar': 'MM',
    'Namibia': 'NA',
    'Nauru': 'NR',
    'Nepal': 'NP',
    'Netherlands': 'NL',
    'New Zealand': 'NZ',
    'Nicaragua': 'NI',
    'Niger': 'NE',
    'Nigeria': 'NG',
    'North Korea': 'KP',
    'North Macedonia': 'MK',
    'Norway': 'NO',
    'Oman': 'OM',
    'Pakistan': 'PK',
    'Palau': 'PW',
    'Palestine State': 'PS',
    'Panama': 'PA',
    'Papua New Guinea': 'PG',
    'Paraguay': 'PY',
    'Peru': 'PE',
    'Philippines': 'PH',
    'Poland': 'PL',
    'Portugal': 'PT',
    'Qatar': 'QA',
    'Romania': 'RO',
    'Russia': 'RU',
    'Rwanda': 'RW',
    'Saint Kitts & Nevis': 'KN',
    'Saint Lucia': 'LC',
    'Saint Vincent & the Grenadines': 'VC',
    'Samoa': 'WS',
    'San Marino': 'SM',
    'Sao Tome & Principe': 'ST',
    'Saudi Arabia': 'SA',
    'Senegal': 'SN',
    'Serbia': 'RS',
    'Seychelles': 'SC',
    'Sierra Leone': 'SL',
    'Singapore': 'SG',
    'Slovakia': 'SK',
    'Slovenia': 'SI',
    'Solomon Islands': 'SB',
    'Somalia': 'SO',
    'South Africa': 'ZA',
    'South Korea': 'KR',
    'South Sudan': 'SS',
    'Spain': 'ES',
    'Sri Lanka': 'LK',
    'Sudan': 'SD',
    'Suriname': 'SR',
    'Sweden': 'SE',
    'Switzerland': 'CH',
    'Syria': 'SY',
    'Tajikistan': 'TJ',
    'Tanzania': 'TZ',
    'Thailand': 'TH',
    'Timor-Leste': 'TL',
    'Togo': 'TG',
    'Tonga': 'TO',
    'Trinidad & Tobago': 'TT',
    'Tunisia': 'TN',
    'Turkey': 'TR',
    'Turkmenistan': 'TM',
    'Tuvalu': 'TV',
    'Uganda': 'UG',
    'Ukraine': 'UA',
    'United Arab Emirates': 'AE',
    'United Kingdom': 'GB',
    'United States': 'US',
    'Uruguay': 'UY',
    'Uzbekistan': 'UZ',
    'Vanuatu': 'VU',
    'Vatican City': 'VA',
    'Venezuela': 'VE',
    'Vietnam': 'VN',
    'Yemen': 'YE',
    'Zambia': 'ZM',
    'Zimbabwe': 'ZW',
}


# ======================================================
# 3. FUNCIÓN: OBTENER LOS PRINCIPALES PAÍSES
# ======================================================
def get_top_countries(keyword, n_countries=5, timeframe=TIMEFRAME_GLOBAL):
    """
    1) build_payload(geo='') a nivel mundial para la keyword dada.
    2) interest_by_region(resolution='COUNTRY', inc_low_vol=True).
    3) Filtra para mantener solo los países que figuran en country_code_map.
    4) Devuelve N países con mayor interés.
    """
    # 1) Payload mundial
    pytrends.build_payload(
        kw_list=[keyword],
        timeframe=timeframe,
        geo=''  # Global
    )
    # 2) Interest por región (País)
    interest_df = pytrends.interest_by_region(resolution='COUNTRY', inc_low_vol=True)
    
    if keyword not in interest_df.columns:
        print(f"[WARNING] No hay columnas para '{keyword}' en interest_by_region. Retorno vacío.")
        return pd.Series([], dtype=float)
    
    # 3) Filtrar: solo nombres de país que aparecen en el diccionario
    valid_countries = set(country_code_map.keys())  # nombres que reconoces
    interest_df = interest_df[interest_df.index.isin(valid_countries)]
    
    # Ordenar de mayor a menor
    top_series = interest_df[keyword].sort_values(ascending=False).head(n_countries)
    # Filtrar > 0
    top_series = top_series[top_series > 0]
    
    return top_series


# ======================================================
# 4. FUNCIÓN: ANALIZAR RELATED TOPICS POR PAÍS
# ======================================================
def analyze_related_topics_by_country(keyword, countries_series, timeframe=TIMEFRAME_GLOBAL):
    """
    1) Itera sobre el nombre de cada país en countries_series.index (ej: 'Brazil').
    2) Convierte a código ISO con country_code_map.
    3) build_payload(geo=...), obtiene related_topics().
    4) Guarda CSV de 'top' y 'rising'.
    """
    for country_name in countries_series.index:
        # Mapeo a código ISO
        code = country_code_map.get(country_name)
        if not code:
            print(f"[WARNING] Sin mapeo para '{country_name}'. Saltando...")
            continue
        
        print(f"\nAnalizando '{keyword}' en '{country_name}' (geo='{code}').")
        
        # build_payload con geo = el código
        pytrends.build_payload(
            kw_list=[keyword],
            timeframe=timeframe,
            geo=code
        )
        
        # Manejo de excepciones en related_topics
        try:
            related_topics_dict = pytrends.related_topics()
        except IndexError:
            print(f"[WARNING] Google no devolvió 'related topics' para '{keyword}' en '{country_name}' ({code}).")
            continue
        
        if keyword not in related_topics_dict:
            print(f"[WARNING] No hay datos en related_topics_dict para '{keyword}' en '{country_name}' ({code}).")
            continue
        
        # -------------------
        # TOP
        # -------------------
        if 'top' in related_topics_dict[keyword] and not related_topics_dict[keyword]['top'].empty:
            df_top = related_topics_dict[keyword]['top']
            csv_top_path = os.path.join(DATA_FOLDER, f"related_topics_top_{keyword}_{code}.csv")
            df_top.to_csv(csv_top_path, index=False)
            print(f"TOP topics guardados en: {csv_top_path}")
            print("Vista previa:\n", df_top.head(3))
        else:
            print(f"Sin datos 'top' en {country_name} para '{keyword}'")
        
        # -------------------
        # RISING
        # -------------------
        if 'rising' in related_topics_dict[keyword] and not related_topics_dict[keyword]['rising'].empty:
            df_rising = related_topics_dict[keyword]['rising']
            csv_rising_path = os.path.join(DATA_FOLDER, f"related_topics_rising_{keyword}_{code}.csv")
            df_rising.to_csv(csv_rising_path, index=False)
            print(f"RISING topics guardados en: {csv_rising_path}")
            print("Vista previa:\n", df_rising.head(3))
        else:
            print(f"Sin datos 'rising' en {country_name} para '{keyword}'")
        
        # Pausa para no saturar Google
        time.sleep(5)


# ======================================================
# 5. CÓDIGO PRINCIPAL
# ======================================================
if __name__ == "__main__":
    for kw in keywords:
        print(f"\n========================")
        print(f" Procesando keyword: {kw}")
        print(f"========================")

        # 1) Obtener top países
        top_countries = get_top_countries(keyword=kw, n_countries=5, timeframe=TIMEFRAME_GLOBAL)
        if top_countries.empty:
            print(f"[WARNING] No se encontraron países con interés (>0) para '{kw}'. Siguiente keyword.")
            continue
        
        print(f"Top {len(top_countries)} países con mayor interés para '{kw}':\n{top_countries}")
        
        # 2) Analizar related topics por país
        analyze_related_topics_by_country(keyword=kw, countries_series=top_countries, timeframe=TIMEFRAME_GLOBAL)
        
        # Pausa entre keywords
        time.sleep(5)


 Procesando keyword: shopping
Top 5 países con mayor interés para 'shopping':
geoName
Brazil               100
Uruguay               73
Trinidad & Tobago     72
Seychelles            56
Paraguay              53
Name: shopping, dtype: int64

Analizando 'shopping' en 'Brazil' (geo='BR').
[WARNING] Google no devolvió 'related topics' para 'shopping' en 'Brazil' (BR).

Analizando 'shopping' en 'Uruguay' (geo='UY').
[WARNING] Google no devolvió 'related topics' para 'shopping' en 'Uruguay' (UY).

Analizando 'shopping' en 'Trinidad & Tobago' (geo='TT').
[WARNING] Google no devolvió 'related topics' para 'shopping' en 'Trinidad & Tobago' (TT).

Analizando 'shopping' en 'Seychelles' (geo='SC').
[WARNING] Google no devolvió 'related topics' para 'shopping' en 'Seychelles' (SC).

Analizando 'shopping' en 'Paraguay' (geo='PY').
[WARNING] Google no devolvió 'related topics' para 'shopping' en 'Paraguay' (PY).

 Procesando keyword: fitness
[WARNING] No se encontraron países con interés (>0) para '

In [13]:
print("Índice (geoName) de top_countries:", top_countries.index.tolist())

Índice (geoName) de top_countries: ['Brazil', 'Uruguay', 'Trinidad & Tobago', 'Seychelles', 'Paraguay']
